<h1 align="center">---- Data Science ----</h1>

----

**0) Packages**

In [8]:
using Pkg
Pkg.add("GLMNet")
Pkg.add("RDatasets")
Pkg.add("MLBase")
Pkg.add("Plots")
Pkg.add("DecisionTree")
Pkg.add("Distances")
Pkg.add("NearestNeighbors")
Pkg.add("Random")
Pkg.add("LinearAlgebra")
Pkg.add("DataStructures")
Pkg.add("LIBSVM")

   Resolving package versions...
  No Changes to `C:\Users\gabri\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\gabri\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\gabri\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\gabri\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\gabri\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\gabri\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\gabri\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\gabri\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\gabri\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\gabri\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\gabri\.julia\environments\v1.8\Project.toml`
  No Changes to 

In [1]:
using GLMNet
using RDatasets
using MLBase
using Plots
using DecisionTree
using Distances
using NearestNeighbors
using Random
using LinearAlgebra
using DataStructures
using LIBSVM

----

**1) Evaluation Metric**

In [19]:
# Accuracy = sum(y' + y) / len(y)
find_accuracy(predicted_y, real_y) = sum(predicted_y.==real_y)  / length(real_y)

find_accuracy (generic function with 1 method)

----

**2) Inspecting Dataset**

In [3]:
df = dataset("datasets", "iris")

println("Shape: $(size(df))")
println("Features: $(names(df))")
println("-*-*-*-*-")

df

Shape: (150, 5)
Features: ["SepalLength", "SepalWidth", "PetalLength", "PetalWidth", "Species"]
-*-*-*-*-


Row,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
,Float64,Float64,Float64,Float64,Cat…
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa
7,4.6,3.4,1.4,0.3,setosa
8,5.0,3.4,1.5,0.2,setosa
9,4.4,2.9,1.4,0.2,setosa


In [4]:
describe(df)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,SepalLength,5.84333,4.3,5.8,7.9,0,Float64
2,SepalWidth,3.05733,2.0,3.0,4.4,0,Float64
3,PetalLength,3.758,1.0,4.35,6.9,0,Float64
4,PetalWidth,1.19933,0.1,1.3,2.5,0,Float64
5,Species,,setosa,,virginica,0,"CategoricalValue{String, UInt8}"


----

**2) Preparing the Datas**

In [27]:
# Splitting up between feature (X) and target (y)
X = Matrix(df[:, 1:4])

labels = df[:, 5]
labels_map = labelmap(labels)
y = labelencode(labels_map, labels)

150-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3

In [28]:
# Function to split dataset into training and validation
function perclass_splits(y, at)
    uids = unique(y)
    keepids = []
    for ui in uids
        curids = findall(y.==ui)
        rowids = randsubseq(curids, at)
        push!(keepids, rowids...)
    end
    return keepids
end

perclass_splits (generic function with 1 method)

In [29]:
# Splitting up train and validation datasets
train_df = perclass_splits(y, 0.7)
valid_df = setdiff(1:length(y), train_df)

46-element Vector{Any}:
   3
   8
  10
  12
  14
  17
  20
  27
  29
  31
  45
  46
  49
   ⋮
 115
 117
 119
 124
 125
 127
 128
 138
 139
 140
 144
 147

In [30]:
# Function to convert predicted values into labels
assign_class(predicted_value) = argmin(abs.(predicted_value .- [1, 2, 3]))

assign_class (generic function with 1 method)

----

**3.0) Lasso Method**

In [39]:
# training the model and choosing the best lambda to predict with
path = glmnet(X[train_df, :], y[train_df]) # creating train df (X and y)
cv = glmnetcv(X[train_df, :], y[train_df]) # creating cross-validation
lambda = path.lambda[argmin(cv.meanloss)] # getting the best lambda, that is, the one with the smallest mean loss

path = glmnet(X[train_df, :], y[train_df], lambda=[lambda]) # training the perfect model

#=
    \ df >> best dataset used over cross-validation step
    \ pct_dev >> accuracy
    \ λ (lambda) >> mean loss score
=#

Least Squares GLMNet Solution Path (1 solutions for 4 predictors in 79 passes):
─────────────────────────────
     df   pct_dev           λ
─────────────────────────────
[1]   4  0.940624  0.00106932
─────────────────────────────

In [40]:
# Making predictions, labelling the thm and getting the
# validation accuracy
q = X[valid_df, :];
predictions_lasso = GLMNet.predict(path, q)

class_predictions_lasso = assign_class.(predictions_lasso)
find_accuracy(class_predictions_lasso, y[valid_df])

0.9782608695652174

----

**3.1) Ridge Method**

In [42]:
path = glmnet(X[train_df, :], y[train_df], alpha=0)
cv = glmnetcv(X[train_df, :], y[train_df], alpha=0)
lambda = path.lambda[argmin(cv.meanloss)]

path = glmnet(X[train_df, :], y[train_df], alpha=0, lambda=[lambda]);

In [43]:
q = X[valid_df, :]
predictions_ridge = GLMNet.predict(path, q)
class_predictions_ridge = assign_class.(predictions_ridge)

find_accuracy(class_predictions_ridge, y[valid_df])

0.9782608695652174

----

**3.2) Elastic Net Method**

In [44]:
path = glmnet(X[train_df, :], y[train_df], alpha=0.5)
cv = glmnetcv(X[train_df, :], y[train_df], alpha=0.5)
lambda = path.lambda[argmin(cv.meanloss)]

path = glmnet(X[train_df, :], y[train_df], alpha=0.5, lambda=[lambda]);

In [45]:
q = X[valid_df, :]
predictions_EN = GLMNet.predict(path, q)
class_predictions_EN = assign_class.(predictions_ridge)

find_accuracy(class_predictions_EN, y[valid_df])

0.9782608695652174

----

**3.3) Decision Trees**

In [46]:
# Creating and Training the model
model = DecisionTreeClassifier(max_depth=2)
DecisionTree.fit!(model, X[train_df, :], y[train_df])

DecisionTreeClassifier
max_depth:                2
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  [1, 2, 3]
root:                     Decision Tree
Leaves: 3
Depth:  2

In [47]:
q = X[valid_df, :]
predictions_DT = DecisionTree.predict(model, q)
find_accuracy(predictions_DT, y[valid_df])

0.8913043478260869

----

**3.4) Random Forests**

In [49]:
model = RandomForestClassifier(n_trees=20)
DecisionTree.fit!(model, X[train_df, :], y[train_df])

RandomForestClassifier
n_trees:             20
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           -1
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             [1, 2, 3]
ensemble:            Ensemble of Decision Trees
Trees:      20
Avg Leaves: 5.15
Avg Depth:  3.6

In [50]:
q = X[valid_df, :]
predictions_RF = DecisionTree.predict(model, q)
find_accuracy(predictions_RF, y[valid_df])

0.9130434782608695

----

**3.5) Nearest Neighbor Method**

In [60]:
# Splitting up datas
X_train = X[train_df, :]
y_train = y[train_df]
kdtree = KDTree(X_train')

X_valid = X[valid_df, :]

46×4 Matrix{Float64}:
 4.7  3.2  1.3  0.2
 5.0  3.4  1.5  0.2
 4.9  3.1  1.5  0.1
 4.8  3.4  1.6  0.2
 4.3  3.0  1.1  0.1
 5.4  3.9  1.3  0.4
 5.1  3.8  1.5  0.3
 5.0  3.4  1.6  0.4
 5.2  3.4  1.4  0.2
 4.8  3.1  1.6  0.2
 5.1  3.8  1.9  0.4
 4.8  3.0  1.4  0.3
 5.3  3.7  1.5  0.2
 ⋮              
 5.8  2.8  5.1  2.4
 6.5  3.0  5.5  1.8
 7.7  2.6  6.9  2.3
 6.3  2.7  4.9  1.8
 6.7  3.3  5.7  2.1
 6.2  2.8  4.8  1.8
 6.1  3.0  4.9  1.8
 6.4  3.1  5.5  1.8
 6.0  3.0  4.8  1.8
 6.9  3.1  5.4  2.1
 6.8  3.2  5.9  2.3
 6.3  2.5  5.0  1.9

In [61]:
# Calculating Distances
indexes, distances = knn(kdtree, X_valid', 5, true)

([[36, 3, 9, 6, 33], [30, 1, 12, 4, 20], [25, 2, 9, 19, 21], [21, 18, 6, 30, 28], [29, 33, 7, 36, 3], [8, 24, 5, 15, 27], [15, 35, 4, 12, 1], [17, 34, 30, 12, 1], [20, 1, 30, 12, 31], [21, 25, 3, 19, 9]  …  [86, 73, 95, 75, 90], [93, 78, 102, 54, 46], [84, 79, 97, 72, 100], [93, 46, 104, 55, 50], [104, 50, 93, 42, 46], [71, 102, 88, 78, 77], [50, 104, 55, 46, 42], [79, 101, 98, 84, 97], [84, 100, 97, 72, 70], [78, 93, 99, 83, 102]], [[0.14142135623730978, 0.24494897427831802, 0.26457513110645897, 0.264575131106459, 0.2999999999999998], [0.09999999999999964, 0.17320508075688762, 0.1999999999999999, 0.22360679774997916, 0.22360679774997916], [0.1, 0.17320508075688784, 0.17320508075688815, 0.19999999999999993, 0.26457513110645925], [0.22360679774997858, 0.2999999999999998, 0.3000000000000002, 0.31622776601683783, 0.31622776601683833], [0.244948974278318, 0.31622776601683816, 0.34641016151377546, 0.4795831523312718, 0.519615242270663], [0.3464101615137753, 0.38729833462074176, 0.3999999999

In [64]:
# Making Predictions
c = y_train[hcat(indexes...)]
possible_labels = map(i->counter(c[:, i]), 1:size(c,2))
predictions_NN = map(i->parse(Int, string(argmax(DataFrame(possible_labels[i])[1,:]))), 1:size(c,2))

find_accuracy(predictions_NN, y[valid_df])

LoadError: ArgumentError: All column names must be either Symbols or strings (mixing is not allowed)